## 学習目標

1. なぜAttentionには位置情報が必要なのかを理解する
2. Sin/Cos関数を使った位置エンコーディングの仕組みを理解する
3. Position Encodingを実装して可視化する
4. 位置情報がどのように埋め込みに加算されるかを確認する

## 1. なぜ位置情報が必要か？

### 問題: Attentionは順序を考慮しない

Self-AttentionやMulti-Head Attentionは、入力の**順序（位置）を考慮しません**。

例えば以下の2つの文:
- 「犬が猫を追いかけた」
- 「猫が犬を追いかけた」

単語の順序を入れ替えても、Attention機構は同じ出力を返してしまいます（単語の集合としてのみ見る）。

### 解決策: Position Encoding

各単語の**位置情報**を埋め込みベクトルに加算することで、モデルに順序情報を与えます:

```
最終的な入力 = 単語埋め込み + 位置エンコーディング
```

これにより、同じ単語でも「文の先頭」と「文の中間」では異なる表現になります。

## 2. 必要なライブラリのインポート

In [ ]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math

# 日本語フォント設定
plt.rcParams['font.sans-serif'] = ['Arial Unicode MS', 'Hiragino Sans']
plt.rcParams['axes.unicode_minus'] = False

# シード値設定
torch.manual_seed(42)
np.random.seed(42)

print(f"PyTorch version: {torch.__version__}")

## 3. Position Encodingの数式

Transformerの元論文 "Attention is All You Need" では、Sin/Cos関数を使った位置エンコーディングが提案されています。

### 数式

位置 $pos$ の次元 $i$ に対して:

$$
PE_{(pos, 2i)} = \sin\left(\frac{pos}{10000^{2i/d_{\text{model}}}}\right)
$$

$$
PE_{(pos, 2i+1)} = \cos\left(\frac{pos}{10000^{2i/d_{\text{model}}}}\right)
$$

### パラメータ

- $pos$: 位置（0, 1, 2, ...）
- $i$: 次元のインデックス
- $d_{\text{model}}$: モデルの次元数
- 偶数次元にはsin、奇数次元にはcosを使用

### なぜこの式？

1. **周期性**: 異なる周波数のsin/cos波を使うことで、位置の相対的な関係を捉える
2. **外挿可能**: 学習時より長い系列にも対応可能
3. **線形変換で相対位置を表現**: $PE_{pos+k}$ が $PE_{pos}$ の線形結合で表現できる

## 4. Position Encodingの実装

In [ ]:
class PositionalEncoding(nn.Module):
    """
    位置エンコーディング層
    
    Sin/Cos関数を使って各位置に固有のベクトルを生成します。
    
    Args:
        d_model: モデルの次元数
        max_len: サポートする最大系列長
        dropout: ドロップアウト率
    """
    def __init__(self, d_model, max_len=5000, dropout=0.1):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)
        
        # 位置エンコーディング行列を作成
        # shape: [max_len, d_model]
        pe = torch.zeros(max_len, d_model)
        
        # 位置インデックス: [0, 1, 2, ..., max_len-1]
        # shape: [max_len, 1]
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        
        # 分母の計算: 10000^(2i/d_model)
        # shape: [d_model/2]
        div_term = torch.exp(
            torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model)
        )
        
        # 偶数次元にはsin
        pe[:, 0::2] = torch.sin(position * div_term)
        
        # 奇数次元にはcos
        pe[:, 1::2] = torch.cos(position * div_term)
        
        # バッチ次元を追加: [1, max_len, d_model]
        pe = pe.unsqueeze(0)
        
        # パラメータではなくバッファとして登録（学習しない）
        self.register_buffer('pe', pe)
    
    def forward(self, x):
        """
        入力に位置エンコーディングを加算
        
        Args:
            x: 入力テンソル [batch_size, seq_len, d_model]
        Returns:
            位置情報が加算された入力 [batch_size, seq_len, d_model]
        """
        # 入力の系列長分だけ位置エンコーディングを取得して加算
        x = x + self.pe[:, :x.size(1), :]
        return self.dropout(x)

## 5. Position Encodingの可視化

In [ ]:
# パラメータ設定
d_model = 128
max_len = 100

# Position Encodingを作成
pos_encoding = PositionalEncoding(d_model, max_len, dropout=0.0)

# 位置エンコーディング行列を取得
pe_matrix = pos_encoding.pe[0].numpy()  # [max_len, d_model]

print(f"位置エンコーディング行列の形状: {pe_matrix.shape}")
print(f"  - 各行: 1つの位置のエンコーディング")
print(f"  - 各列: 1つの特徴次元")

# ヒートマップで可視化
plt.figure(figsize=(12, 8))
plt.imshow(pe_matrix, cmap='RdBu_r', aspect='auto')
plt.colorbar(label='値')
plt.xlabel('次元 (d_model)', fontsize=12)
plt.ylabel('位置 (pos)', fontsize=12)
plt.title('Position Encoding のヒートマップ', fontsize=14, pad=20)
plt.tight_layout()
plt.savefig('position_encoding_heatmap.png', dpi=150, bbox_inches='tight')
plt.show()

print("\n💡 観察ポイント:")
print("  - 縦の縞模様: 各次元が異なる周波数の波を持つ")
print("  - 左側（低次元）: 低周波（ゆっくり変化）")
print("  - 右側（高次元）: 高周波（速く変化）")

## 6. 特定の位置のエンコーディングを詳しく見る

In [ ]:
# 複数の位置のエンコーディングをプロット
positions = [0, 10, 50, 99]

plt.figure(figsize=(14, 8))

for pos in positions:
    plt.plot(pe_matrix[pos], label=f'位置 {pos}', alpha=0.7, linewidth=2)

plt.xlabel('次元', fontsize=12)
plt.ylabel('エンコーディング値', fontsize=12)
plt.title('異なる位置のPosition Encoding', fontsize=14)
plt.legend(fontsize=10)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig('position_encoding_by_pos.png', dpi=150, bbox_inches='tight')
plt.show()

print("💡 各位置が固有のパターンを持っています")
print("   これにより、モデルは位置を区別できます")

## 7. 特定の次元の変化を見る

In [ ]:
# いくつかの次元について、位置による変化をプロット
dimensions = [0, 1, 10, 20, 50, 100]

fig, axes = plt.subplots(2, 3, figsize=(15, 8))
axes = axes.flatten()

for idx, dim in enumerate(dimensions):
    axes[idx].plot(pe_matrix[:, dim], linewidth=2)
    axes[idx].set_title(f'次元 {dim}', fontsize=11)
    axes[idx].set_xlabel('位置')
    axes[idx].set_ylabel('値')
    axes[idx].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('position_encoding_by_dim.png', dpi=150, bbox_inches='tight')
plt.show()

print("💡 観察:")
print("  - 低い次元（0, 1）: 長い周期のsin/cos波")
print("  - 高い次元（50, 100）: 短い周期の波")
print("  - 複数の周波数を組み合わせることで、位置を一意に識別")

## 8. 実際の使用例: 単語埋め込み + Position Encoding

In [ ]:
# パラメータ
batch_size = 2
seq_len = 10
d_model = 64

# ランダムな単語埋め込みを生成（実際にはEmbedding層から得られる）
word_embeddings = torch.randn(batch_size, seq_len, d_model)

print("単語埋め込みのみ:")
print(f"  形状: {word_embeddings.shape}")
print(f"  最初のサンプル、最初の単語の埋め込み（一部）:")
print(f"  {word_embeddings[0, 0, :10].numpy()}")

# Position Encodingを適用
pos_enc = PositionalEncoding(d_model, dropout=0.0)
embeddings_with_pos = pos_enc(word_embeddings)

print("\n単語埋め込み + Position Encoding:")
print(f"  形状: {embeddings_with_pos.shape}")
print(f"  最初のサンプル、最初の単語の埋め込み（一部）:")
print(f"  {embeddings_with_pos[0, 0, :10].detach().numpy()}")

# 位置エンコーディングの寄与を確認
position_contribution = pos_enc.pe[0, :seq_len, :]
print("\n位置エンコーディングの寄与（一部）:")
print(f"  {position_contribution[0, :10].numpy()}")

print("\n💡 単語埋め込みに位置情報が加算され、")
print("   同じ単語でも位置によって異なる表現になります！")

## 9. 位置エンコーディングの効果を確認

In [ ]:
# 同じ単語が異なる位置にある場合を想定
# すべての位置で同じ埋め込みベクトルを使用
same_word_embedding = torch.randn(1, d_model)
repeated_embeddings = same_word_embedding.repeat(1, seq_len, 1)

print("位置エンコーディングなし（同じ単語）:")
print(f"  すべての位置で同じベクトル")
print(f"  位置0と位置5の差: {(repeated_embeddings[0, 0] - repeated_embeddings[0, 5]).abs().sum().item():.6f}")

# Position Encodingを適用
with_pos = pos_enc(repeated_embeddings)

print("\n位置エンコーディングあり（同じ単語）:")
print(f"  各位置で異なるベクトル")
print(f"  位置0と位置5の差: {(with_pos[0, 0] - with_pos[0, 5]).abs().sum().item():.6f}")

# コサイン類似度で確認
cos_sim_without = nn.functional.cosine_similarity(
    repeated_embeddings[0, 0].unsqueeze(0),
    repeated_embeddings[0, 5].unsqueeze(0)
)
cos_sim_with = nn.functional.cosine_similarity(
    with_pos[0, 0].unsqueeze(0),
    with_pos[0, 5].unsqueeze(0)
)

print(f"\nコサイン類似度:")
print(f"  位置エンコーディングなし: {cos_sim_without.item():.6f} (1.0 = 完全に同じ)")
print(f"  位置エンコーディングあり: {cos_sim_with.item():.6f}")

print("\n💡 位置エンコーディングにより、同じ単語でも")
print("   位置が異なれば異なる表現になることが確認できました！")

## 10. まとめ

### Position Encodingの役割

1. **位置情報の付与**: Attention機構に順序情報を与える
2. **固定パターン**: 学習不要の固定的なsin/cos波を使用
3. **加算方式**: 単語埋め込みに加算するだけのシンプルな実装

### Sin/Cos関数を使う理由

1. **周期性**: 異なる周波数で位置の相対関係を捉える
2. **外挿可能**: 学習時より長い系列にも対応
3. **計算効率**: 事前計算してバッファに保存できる

### 実装のポイント

- 偶数次元にsin、奇数次元にcosを使用
- `register_buffer`で学習しないパラメータとして登録
- ドロップアウトで正則化

### 次のステップ

- Feed Forward Network: Attention後の非線形変換
- Layer Normalization: 層の正規化
- Encoder Block: Self-Attention + FFN + 残差接続
- Decoder Block: Masked Attention + Cross Attention

Position Encodingにより、Transformerは単語の順序を理解できるようになりました！